<a href="https://colab.research.google.com/github/uyenphuong01/PBL3/blob/main/Yellow_box_junction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ultralytics easyocr

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import deque
import easyocr
from datetime import datetime
from pathlib import Path
import math
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

#CẤU HÌNH KẾT NỐI API - RENDER PRODUCTION
import requests
from datetime import datetime
import os

#API PRODUCTION URL
API_URL = "https://traffic-violation-system-ei12.onrender.com/api"

# Test kết nối
print("\n" + "="*60)
print("TESTING API CONNECTION")
print("="*60)
try:
    response = requests.get(f"{API_URL}/health", timeout=10)
    if response.status_code == 200:
        print("API đang hoạt động!")
        print(f"Connected to: {API_URL}")
        print(f"Response: {response.json()}")
    else:
        print(f"API trả về status: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("KHÔNG THỂ KẾT NỐI API!")
    print("Kiểm tra Render service có đang chạy không")
except Exception as e:
    print(f"Lỗi: {str(e)}")
print("="*60 + "\n")


In [ ]:

# TEST KẾT NỐI API TRƯỚC KHI XỬ LÝ VIDEO
import requests
API_URL = "https://traffic-violation-system-ei12.onrender.com/api"
print("="*70)
print("TESTING API CONNECTION")
print("="*70 + "\n")

try:
    # Test 1: Health check
    print("1. Testing health endpoint...")
    response = requests.get(f"{API_URL}/health", timeout=10)
    if response.status_code == 200:
        print("API is online!")
        print(f"Response: {response.json()}")
    else:
        print(f"Status: {response.status_code}")
    # Test 2: Statistics
    print("\n2. Testing statistics endpoint...")
    response = requests.get(f"{API_URL}/statistics", timeout=10)
    if response.status_code == 200:
        stats = response.json()
        print("Statistics endpoint OK!")
        print(f"Total vehicles: {stats['total_vehicles']}")
        print(f"Total violations: {stats['total_violations']}")

    # Test 3: Check có xe nào trong database không
    print("\n3. Checking vehicles in database...")
    response = requests.get(f"{API_URL}/vehicles", timeout=10)
    if response.status_code == 200:
        vehicles = response.json()
        print(f"Found {len(vehicles)} vehicles in database")
        if len(vehicles) > 0:
            print("Sample vehicles:")
            for v in vehicles[:3]:
                print(f"      - {v['plate_number']}: {v['owner_name']}")
        else:
            print("No vehicles in database yet!")
            print("Add vehicles at: https://traffic-violation-system-ei12.onrender.com")

    print("\n" + "="*70)
    print(" API CONNECTION TEST PASSED!")
    print("="*70 + "\n")

except requests.exceptions.ConnectionError:
    print("\n CANNOT CONNECT TO API!")
    print("Please check:")
    print("1. Internet connection")
    print("2. API URL is correct")
    print("3. Render service is not sleeping")
    print("\nTrying to wake up service...")
    print("Please wait 30 seconds and run this cell again...")

except Exception as e:
    print(f"\n Error: {str(e)}")

In [ ]:

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="HEvHQ2IwQILi6IvwX4H6")
project = rf.workspace("trafficlightdetection-kdc1y").project("box_junction-kfqy9")
version = project.version(7)
dataset = version.download("yolov8")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to box_junction-7 in yolov8:: 100%|██████████| 688/688 [00:00<00:00, 1477.49it/s]


In [ ]:

DATA_YAML_PATH = "/content/box_junction-7/data.yaml"

File cấu hình dataset nằm ở: /content/box_junction-7/data.yaml


In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8s-seg.pt')

results = model.train(
    data=DATA_YAML_PATH,
    epochs=100,
    imgsz=640,
    batch=16,
    name='yellow_box_seg_model'
)

Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/box_junction-7/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yellow_box_seg_model, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspectiv

In [ ]:
!cp -r /content/runs /content/drive/MyDrive/ALPR_YOLOv8_Project

In [ ]:

import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package],
                            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except:
        return False

for pkg in ['scipy', 'ultralytics', 'easyocr']:
    try:
        __import__(pkg)
    except ImportError:
        print(f"   Installing {pkg}...")
        install_package(pkg)

PADDLE_AVAILABLE = False
try:
    import paddle
    from paddleocr import PaddleOCR
    PADDLE_AVAILABLE = True
    print("PaddleOCR available - will use it")
except:
    print(" PaddleOCR not available - will use EasyOCR")

print("All packages ready!\n")

import cv2
import numpy as np
import torch
import re
from ultralytics import YOLO
from collections import deque, defaultdict
import easyocr
from datetime import datetime
from pathlib import Path
import math

import torch.serialization
from ultralytics.nn.tasks import DetectionModel, SegmentationModel

torch.serialization.add_safe_globals([DetectionModel, SegmentationModel])
print("PyTorch safe globals configured")


In [ ]:
# CẤU HÌNH HỆ THỐNG
class Config:
    # Model paths
    # Model SEGMENTATION Yellow Box Junction (đã train)
    YELLOW_BOX_SEG_MODEL = "/content/drive/MyDrive/PBL3/runs/segment/yellow_box_seg_model/weights/best.pt"
    # Model DETECTION Biển số (đã train)
    PLATE_MODEL_PATH = "/content/drive/MyDrive/PBL3/license_plate_yolov83/weights/best.pt"

    # Vehicle detection
    VEHICLE_CLASSES = {
        2: "car",
        3: "motorcycle",
        5: "bus",
        7: "truck"
    }
    # Detection thresholds
    CONF_THRESHOLD = 0.35
    YELLOW_BOX_CONF = 0.5
    PLATE_CONF = 0.3
    # Tracking parameters
    MAX_TRACKING_DISTANCE = 120
    STATIONARY_THRESHOLD = 10
    VIOLATION_TIME = 3.0
    MAX_LOST_FRAMES = 40
    # Video settings
    FPS = 30
    OUTPUT_DIR = "/content/violations_output"
    # OCR settings
    USE_PADDLEOCR = PADDLE_AVAILABLE
    # Location
    LOCATION = "Ngã tư Hùng Vương - Lê Duẩn, Đà Nẵng"

In [ ]:
# YELLOW BOX JUNCTION DETECTOR
class YellowBoxDetector:
    def __init__(self, model_path):
        print(f"Loading Yellow Box Model...")
        self.model = YOLO(model_path)
        self.current_mask = None
        self.current_polygon = None
        self.box_area = 0
        print("Yellow Box Model Ready!")

    def detect(self, frame):
        results = self.model(frame, conf=Config.YELLOW_BOX_CONF, verbose=False)[0]

        self.current_mask = None
        self.current_polygon = None
        self.box_area = 0

        if results.masks is not None and len(results.masks) > 0:
            best_idx = 0
            if len(results.boxes) > 1:
                confidences = results.boxes.conf.cpu().numpy()
                best_idx = np.argmax(confidences)

            mask = results.masks[best_idx]
            mask_data = mask.data[0].cpu().numpy()
            h, w = frame.shape[:2]
            self.current_mask = cv2.resize(mask_data, (w, h))
            self.current_mask = (self.current_mask > 0.5).astype(np.uint8)
            self.box_area = np.sum(self.current_mask)

            if mask.xy is not None and len(mask.xy) > 0:
                self.current_polygon = mask.xy[best_idx].astype(np.int32)

        return self.current_mask, self.current_polygon

    def draw(self, frame):
        if self.current_mask is not None and self.current_polygon is not None:
            cv2.polylines(frame, [self.current_polygon], True, (0, 255, 255), 4)

            if len(self.current_polygon) > 0:
                x, y = self.current_polygon[0]
                text = "YELLOW BOX JUNCTION"
                (text_w, text_h), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
                cv2.rectangle(frame, (int(x), int(y) - text_h - 15),
                            (int(x) + text_w + 10, int(y) - 5),
                            (0, 255, 255), -1)
                cv2.putText(frame, text, (int(x) + 5, int(y) - 10),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
        return frame

    def is_inside(self, bbox):
        if self.current_mask is None:
            return False

        h, w = self.current_mask.shape
        x1, y1, x2, y2 = bbox

        points = [
            (int((x1 + x2) / 2), int(y2)),
            (int(x1 + (x2-x1)*0.25), int(y2)),
            (int(x1 + (x2-x1)*0.75), int(y2)),
            (int((x1 + x2) / 2), int((y1 + y2) / 2)),
            (int((x1 + x2) / 2), int(y1 + (y2-y1)*0.3)),
        ]

        inside_count = 0
        for px, py in points:
            if 0 <= px < w and 0 <= py < h:
                if self.current_mask[py, px] == 1:
                    inside_count += 1

        return inside_count >= 2


In [ ]:
# VEHICLE TRACKING
class VehicleInfo:
    def __init__(self, track_id, bbox, frame_num, vehicle_type):
        self.track_id = track_id
        self.vehicle_type = vehicle_type
        self.bbox = bbox

        self.trajectory = deque(maxlen=60)
        center = self.get_center(bbox)
        self.trajectory.append(center)
        self.last_pos = center

        self.in_box = False
        self.was_moving = True
        self.stationary_frames = 0
        self.stationary_start_frame = None

        self.is_violation = False
        self.violation_captured = False
        self.plate_number = "N/A"

        self.first_seen = frame_num
        self.last_seen = frame_num
        self.frames_lost = 0

        self.direction = "Unknown"
        self.avg_speed = 0

    @staticmethod
    def get_center(bbox):
        return ((bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2)

    def update(self, bbox, frame_num):
        self.bbox = bbox
        self.last_seen = frame_num
        self.frames_lost = 0

        center = self.get_center(bbox)
        self.trajectory.append(center)

        distance = math.hypot(center[0] - self.last_pos[0], center[1] - self.last_pos[1])

        if len(self.trajectory) >= 10:
            recent = list(self.trajectory)[-10:]
            total_dist = sum(
                math.hypot(recent[i][0] - recent[i-1][0], recent[i][1] - recent[i-1][1])
                for i in range(1, len(recent))
            )
            self.avg_speed = total_dist / 9

        self.update_direction()

        if distance < Config.STATIONARY_THRESHOLD:
            self.stationary_frames += 1
            if self.stationary_start_frame is None:
                self.stationary_start_frame = frame_num
            self.was_moving = False
        else:
            if not self.was_moving:
                self.stationary_frames = 0
                self.stationary_start_frame = None
            self.was_moving = True

        self.last_pos = center

    def update_direction(self):
        if len(self.trajectory) < 20:
            return

        start = self.trajectory[0]
        end = self.trajectory[-1]
        dx = end[0] - start[0]
        dy = end[1] - start[1]
        angle = math.degrees(math.atan2(dy, dx))

        if -45 <= angle < 45:
            self.direction = "East →"
        elif 45 <= angle < 135:
            self.direction = "South ↓"
        elif angle >= 135 or angle < -135:
            self.direction = "West ←"
        else:
            self.direction = "North ↑"

    def check_violation(self, in_box, frame_num, fps):
        self.in_box = in_box

        if in_box and self.stationary_start_frame is not None:
            stopped_time = (frame_num - self.stationary_start_frame) / fps

            if stopped_time >= Config.VIOLATION_TIME and not self.is_violation:
                self.is_violation = True
                print(f" VI PHẠM! ID:{self.track_id} [{self.vehicle_type}] "
                      f"Dừng: {stopped_time:.1f}s | Hướng: {self.direction}")
                return True

        return False

    def get_stopped_time(self, current_frame, fps):
        if self.stationary_start_frame is None:
            return 0
        return (current_frame - self.stationary_start_frame) / fps


# VEHICLE TRACKER
class VehicleTracker:
    def __init__(self):
        self.tracks = {}
        self.next_id = 0

    def update(self, detections, frame_num, yellow_box_detector, fps):
        current_tracks = {}
        matched_detections = set()
        matched_tracks = set()

        for det_idx, detection in enumerate(detections):
            bbox = detection['bbox']
            vehicle_type = detection['type']
            center = VehicleInfo.get_center(bbox)

            best_track_id = None
            min_distance = Config.MAX_TRACKING_DISTANCE

            for track_id, vehicle in self.tracks.items():
                if track_id in matched_tracks or vehicle.vehicle_type != vehicle_type:
                    continue

                distance = math.hypot(center[0] - vehicle.last_pos[0], center[1] - vehicle.last_pos[1])

                if distance < min_distance:
                    min_distance = distance
                    best_track_id = track_id

            if best_track_id is not None:
                vehicle = self.tracks[best_track_id]
                vehicle.update(bbox, frame_num)
                vehicle.check_violation(yellow_box_detector.is_inside(bbox), frame_num, fps)
                current_tracks[best_track_id] = vehicle
                matched_detections.add(det_idx)
                matched_tracks.add(best_track_id)

        for det_idx, detection in enumerate(detections):
            if det_idx in matched_detections:
                continue

            track_id = self.next_id
            self.next_id += 1

            vehicle = VehicleInfo(track_id, detection['bbox'], frame_num, detection['type'])
            vehicle.check_violation(yellow_box_detector.is_inside(detection['bbox']), frame_num, fps)

            self.tracks[track_id] = vehicle
            current_tracks[track_id] = vehicle

            print(f"New vehicle: ID {track_id} [{vehicle.vehicle_type}]")

        for track_id in list(self.tracks.keys()):
            if track_id not in matched_tracks:
                vehicle = self.tracks[track_id]
                vehicle.frames_lost += 1

                if vehicle.frames_lost > Config.MAX_LOST_FRAMES:
                    print(f"Lost track: ID {track_id} [{vehicle.vehicle_type}]")
                    del self.tracks[track_id]

        return current_tracks

In [ ]:
# HYBRID PLATE OCR
class PlateOCR:
    def __init__(self, model_path, use_gpu=True):
        print(f"Loading Plate Detection Model & OCR...")
        self.plate_model = YOLO(model_path)

        # Initialize OCR
        if Config.USE_PADDLEOCR:
            print(f" Using PaddleOCR (GPU: {use_gpu})")
            try:
                self.ocr = PaddleOCR(lang="en", use_angle_cls=True, use_gpu=use_gpu, show_log=False)
                self.ocr_type = "paddle"
            except Exception as e:
                print(f" PaddleOCR failed: {e}")
                print(f" Falling back to EasyOCR...")
                self.ocr = easyocr.Reader(['en'], gpu=use_gpu, verbose=False)
                self.ocr_type = "easy"
        else:
            print(f"   Using EasyOCR (GPU: {use_gpu})")
            self.ocr = easyocr.Reader(['en'], gpu=use_gpu, verbose=False)
            self.ocr_type = "easy"

        print(f"Plate OCR Ready! (Engine: {self.ocr_type.upper()})")

    def crop_plate(self, frame, vehicle_bbox):
        x1, y1, x2, y2 = map(int, vehicle_bbox)
        h, w = frame.shape[:2]

        margin = 20
        x1 = max(0, x1 - margin)
        y1 = max(0, y1 - margin)
        x2 = min(w, x2 + margin)
        y2 = min(h, y2 + margin)

        vehicle_crop = frame[y1:y2, x1:x2]

        if vehicle_crop.size == 0:
            return None, None

        results = self.plate_model(vehicle_crop, conf=Config.PLATE_CONF, verbose=False)[0]

        if not results.boxes or len(results.boxes) == 0:
            return None, None

        best_idx = 0
        if len(results.boxes) > 1:
            confidences = results.boxes.conf.cpu().numpy()
            best_idx = np.argmax(confidences)

        plate_bbox = results.boxes.xyxy[best_idx].cpu().numpy()
        px1, py1, px2, py2 = map(int, plate_bbox)

        plate_img = vehicle_crop[py1:py2, px1:px2]

        return plate_img if plate_img.size > 0 else None, vehicle_crop

    def preprocess_plate(self, plate_img):
        if plate_img is None or plate_img.size == 0:
            return []

        # Resize 4x
        img_large = cv2.resize(plate_img, None, fx=4, fy=4, interpolation=cv2.INTER_CUBIC)

        gray = cv2.cvtColor(img_large, cv2.COLOR_BGR2GRAY)

        # CLAHE
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        gray = clahe.apply(gray)

        # Denoise
        gray = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)

        variants = []

        # OTSU
        _, otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        variants.append(cv2.cvtColor(otsu, cv2.COLOR_GRAY2BGR))

        # Adaptive
        adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                         cv2.THRESH_BINARY, 11, 2)
        variants.append(cv2.cvtColor(adaptive, cv2.COLOR_GRAY2BGR))

        # Enhanced
        variants.append(cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR))

        return variants

    def format_vietnamese_plate(self, raw_text):
        clean = re.sub(r"[^A-Z0-9]", "", raw_text.upper())

        if len(clean) < 7 or len(clean) > 12:
            return None

        patterns = [
            r"^(\d{2})([A-Z])([A-Z0-9])(\d{4,5})$",
            r"^(\d{2})([A-Z]{1,2})(\d{4,6})$",
            r"^(\d{2})([A-Z]{2,3})(\d{4,5})$",
            r"^(\d{2})([A-Z]{1})(\d{1})(\d{4,5})$",
        ]

        for pattern in patterns:
            match = re.match(pattern, clean)
            if match:
                groups = match.groups()
            # Xử lý theo số lượng groups
                if len(groups) == 4:
                    return f"{groups[0]}{groups[1]}{groups[2]}-{groups[3]}"
                elif len(groups) == 3:
                    return f"{groups[0]}{groups[1]}-{groups[2]}"

        return None

    def ocr_paddle(self, img):
        try:
            result = self.ocr.ocr(img, cls=True)

            if not result or not result[0]:
                return None, 0

            texts = []
            confidences = []

            for line in result[0]:
                if line:
                    text_info = line[1]
                    texts.append(text_info[0])
                    confidences.append(text_info[1])

            if not texts:
                return None, 0

            raw_text = "".join(texts)
            avg_conf = sum(confidences) / len(confidences) if confidences else 0

            return raw_text, avg_conf

        except Exception as e:
            return None, 0

    def ocr_easy(self, img):
        try:
            results = self.ocr.readtext(img, detail=1)
            if not results:
                return None, 0
            # Sort by y then x
            results.sort(key=lambda x: (x[0][0][1], x[0][0][0]))
            raw_text = ''.join(text for _, text, _ in results)
            avg_conf = sum(conf for _, _, conf in results) / len(results)
            return raw_text, avg_conf
        except Exception as e:
            return None, 0

    def read_plate(self, frame, bbox):
        plate_img, vehicle_crop = self.crop_plate(frame, bbox)
        if plate_img is None:
            return None, None
        variants = self.preprocess_plate(plate_img)
        if not variants:
            return None, plate_img
        best_plate = None
        max_confidence = 0
        for variant in variants:
            if self.ocr_type == "paddle":
                raw_text, conf = self.ocr_paddle(variant)
            else:
                raw_text, conf = self.ocr_easy(variant)
            if raw_text:
                formatted = self.format_vietnamese_plate(raw_text)
                if formatted and conf > max_confidence:
                    best_plate = formatted
                    max_confidence = conf
        return best_plate, plate_img

In [ ]:
# VIOLATION MANAGER - VỚI API INTEGRATION
import requests
from pathlib import Path
from datetime import datetime
import cv2

class ViolationManager:
    def __init__(self, output_dir, api_url=None):
        self.dir = Path(output_dir)
        self.dir.mkdir(exist_ok=True)
        self.log_path = self.dir / "violations.txt"
        self.violations = []
        # API Configuration
        self.api_url = api_url or "https://traffic-violation-system-ei12.onrender.com/api"
        self.api_enabled = api_url is not None

        with open(self.log_path, 'w', encoding='utf-8') as f:
            f.write(f"{'='*70}\n")
            f.write("YELLOW BOX JUNCTION VIOLATION LOG\n")
            f.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Location: {Config.LOCATION}\n")
            f.write(f"OCR Engine: {Config.USE_PADDLEOCR and 'PaddleOCR' or 'EasyOCR'}\n")
            f.write(f"API: {self.api_url if self.api_enabled else 'DISABLED'}\n")
            f.write(f"{'='*70}\n\n")

    def send_to_api(self, violation_info, vehicle_img_path, plate_img_path):
        try:
            print(f"\n Sending violation to API...")
            violation_data = {
                'plate_number': violation_info['plate_number'],
                'location': violation_info['location'],
                'violation_time': datetime.now().isoformat(),
                'stopped_duration': float(violation_info['stopped_duration'].replace('s', '')),
                'direction': violation_info['direction']
            }
            files = {}
            with open(vehicle_img_path, 'rb') as f:
                vehicle_img_data = f.read()
                files['vehicle_image'] = ('vehicle.jpg', vehicle_img_data, 'image/jpeg')
                if plate_img_path and Path(plate_img_path).exists():
                    with open(plate_img_path, 'rb') as pf:
                        plate_img_data = pf.read()
                        files['plate_image'] = ('plate.jpg', plate_img_data, 'image/jpeg')
                        response = requests.post(
                            f"{self.api_url}/violations",
                            data=violation_data,
                            files=files,
                            timeout=30
                        )
                else:
                    response = requests.post(
                        f"{self.api_url}/violations",
                        data=violation_data,
                        files=files,
                        timeout=30
                    )

            if response.status_code == 201:
                result = response.json()
                print(f"   Sent to API successfully!")
                print(f"   Violation ID: {result['violation']['id']}")
                print(f"   Status: {result['violation']['status']}")

                if result.get('email_sent'):
                    print(f" Email notification sent!")
                    if result['violation'].get('vehicle_info'):
                        print(f" Recipient: {result['violation']['vehicle_info']['email']}")
                else:
                    print(f" Vehicle owner not found in database - No email sent")

                return True, result
            else:
                print(f" API returned status {response.status_code}")
                print(f"   Response: {response.text}")
                return False, None

        except requests.exceptions.ConnectionError:
            print(f"Cannot connect to API at {self.api_url}")
            print(f"Please check:")
            print(f"1. API server is running")
            print(f"2. URL is correct")
            print(f"3. Internet connection is stable")
            return False, None

        except requests.exceptions.Timeout:
            print(f"Request timeout (>30s)")
            print(f"The API might be sleeping (Render free tier)")
            print(f"Try again in a moment...")
            return False, None

        except Exception as e:
            print(f" Error sending to API: {str(e)}")
            import traceback
            traceback.print_exc()
            return False, None

    def save_violation(self, track_id, vehicle, frame, plate_text, plate_img, fps):
        """Lưu vi phạm local VÀ gửi lên API"""
        timestamp = datetime.now()
        vid = f"VL_{timestamp.strftime('%Y%m%d_%H%M%S')}_{track_id}"

        # Crop ảnh xe
        x1, y1, x2, y2 = map(int, vehicle.bbox)
        vehicle_img = frame[y1:y2, x1:x2]

        # Lưu ảnh xe
        vehicle_img_path = self.dir / f"{vid}_vehicle.jpg"
        cv2.imwrite(str(vehicle_img_path), vehicle_img)

        # Lưu ảnh biển số (nếu có)
        plate_img_path = None
        if plate_img is not None:
            plate_img_path = self.dir / f"{vid}_plate.jpg"
            cv2.imwrite(str(plate_img_path), plate_img)

        # Tính thời gian dừng
        stopped_time = vehicle.get_stopped_time(vehicle.last_seen, fps)

        # Tạo violation info
        violation_info = {
            'id': vid,
            'timestamp': timestamp.strftime('%Y-%m-%d %H:%M:%S'),
            'plate_number': plate_text or "KHÔNG ĐỌC ĐƯỢC",
            'vehicle_type': vehicle.vehicle_type,
            'direction': vehicle.direction,
            'stopped_duration': f"{stopped_time:.1f}s",
            'location': Config.LOCATION,
            'vehicle_image': str(vehicle_img_path),
            'plate_image': str(plate_img_path) if plate_img_path else "N/A"
        }

        self.violations.append(violation_info)

        # Ghi log local
        with open(self.log_path, 'a', encoding='utf-8') as f:
            f.write(f"[{vid}]\n")
            f.write(f"  Thời gian: {violation_info['timestamp']}\n")
            f.write(f"  Biển số: {violation_info['plate_number']}\n")
            f.write(f"  Loại xe: {violation_info['vehicle_type']}\n")
            f.write(f"  Hướng: {violation_info['direction']}\n")
            f.write(f"  Thời gian dừng: {violation_info['stopped_duration']}\n")
            f.write(f"  Địa điểm: {violation_info['location']}\n")
            f.write(f"{'-'*70}\n\n")

        print(f"\n Saved violation locally: {vid}")
        print(f"   Plate: {violation_info['plate_number']}")
        print(f"   Vehicle: {violation_info['vehicle_type']}")
        print(f"   Duration: {violation_info['stopped_duration']}")
        print(f"   Location: {violation_info['location']}")

        # GỬI LÊN API SERVER
        if self.api_enabled:
            success, api_result = self.send_to_api(
                violation_info,
                vehicle_img_path,
                plate_img_path
            )
            if success:
                violation_info['api_sent'] = True
                violation_info['api_id'] = api_result['violation']['id']
            else:
                violation_info['api_sent'] = False
                print(f"Violation saved locally but not sent to API")
        else:
            violation_info['api_sent'] = False
            print(f"API integration is disabled")

        return violation_info

In [ ]:
# MAIN PROCESSOR
class MainProcessor:
    def __init__(self, video_path, api_url=None):
        self.cap = cv2.VideoCapture(video_path)
        if not self.cap.isOpened():
            raise ValueError(f"Cannot open video: {video_path}")
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS)) or Config.FPS
        self.frame_num = 0
        print("\n" + "="*70)
        print("INITIALIZING YELLOW BOX VIOLATION DETECTION SYSTEM")
        print("="*70)
        print("\n Loading models...")
        self.vehicle_detector = YOLO("yolov8s.pt")
        self.yellow_box_detector = YellowBoxDetector(Config.YELLOW_BOX_SEG_MODEL)
        self.plate_ocr = PlateOCR(Config.PLATE_MODEL_PATH, use_gpu=True)
        self.tracker = VehicleTracker()
        # Khởi tạo ViolationManager với API
        self.violation_mgr = ViolationManager(
            Config.OUTPUT_DIR,
            api_url=api_url
        )

        print(f"\n All systems ready!")
        print(f" Video FPS: {self.fps}")
        print(f" Location: {Config.LOCATION}")
        print(f" OCR Engine: {self.plate_ocr.ocr_type.upper()}")
        if api_url:
            print(f" API: {api_url}")
        else:
            print(f" API: DISABLED")
        print("="*70 + "\n")

    def process(self, output_path):
        """Xử lý video"""
        w = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, self.fps, (w, h))
        print(f"Processing video: {w}x{h}, {total_frames} frames")
        print(f"Estimated time: {total_frames/self.fps/60:.1f} minutes\n")

        try:
            while True:
                ret, frame = self.cap.read()
                if not ret:
                    break
                self.frame_num += 1
                # Detect Yellow Box
                self.yellow_box_detector.detect(frame)
                frame = self.yellow_box_detector.draw(frame)
                # Detect vehicles
                results = self.vehicle_detector(
                    frame,
                    conf=Config.CONF_THRESHOLD,
                    verbose=False,
                    classes=list(Config.VEHICLE_CLASSES.keys())
                )[0]
                detections = []
                for box in results.boxes:
                    cls_id = int(box.cls[0])
                    if cls_id in Config.VEHICLE_CLASSES:
                        detections.append({
                            'bbox': box.xyxy[0].cpu().numpy(),
                            'type': Config.VEHICLE_CLASSES[cls_id]
                        })

                # Update tracking
                tracks = self.tracker.update(
                    detections,
                    self.frame_num,
                    self.yellow_box_detector,
                    self.fps
                )
                # Draw và xử lý violations
                for track_id, vehicle in tracks.items():
                    x1, y1, x2, y2 = map(int, vehicle.bbox)
                    # Color theo trạng thái
                    if vehicle.is_violation:
                        color = (0, 0, 255)
                        thickness = 3
                    elif vehicle.in_box:
                        color = (0, 165, 255)
                        thickness = 2
                    else:
                        color = (0, 255, 0)
                        thickness = 2
                    # Vẽ bbox
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)

                    # Vẽ trajectory
                    if len(vehicle.trajectory) > 1:
                        pts = list(vehicle.trajectory)
                        for i in range(1, len(pts)):
                            cv2.line(
                                frame,
                                (int(pts[i-1][0]), int(pts[i-1][1])),
                                (int(pts[i][0]), int(pts[i][1])),
                                color, 2
                            )
                    # Label
                    label_parts = [f"ID:{track_id}", f"[{vehicle.vehicle_type}]"]

                    if vehicle.plate_number != "N/A":
                        label_parts.append(vehicle.plate_number)
                    else:
                        label_parts.append(vehicle.direction)

                    if vehicle.in_box:
                        stopped_time = vehicle.get_stopped_time(self.frame_num, self.fps)
                        label_parts.append(f"{stopped_time:.1f}s")

                    label = " ".join(label_parts)

                    # Vẽ label
                    (label_w, label_h), _ = cv2.getTextSize(
                        label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2
                    )
                    cv2.rectangle(
                        frame,
                        (x1, y1 - label_h - 10),
                        (x1 + label_w, y1),
                        color, -1
                    )
                    cv2.putText(
                        frame, label,
                        (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2
                    )

                    # Xử lý vi phạm với PaddleOCR
                    if vehicle.is_violation and not vehicle.violation_captured:
                        print(f"\n Capturing violation for ID {track_id}...")

                        # Đọc biển số bằng PaddleOCR
                        plate_text, plate_img = self.plate_ocr.read_plate(
                            frame, vehicle.bbox
                        )

                        if plate_text:
                            vehicle.plate_number = plate_text
                            print(f"Plate detected: {plate_text}")
                        else:
                            print(f"Could not read plate")

                        # Lưu vi phạm
                        self.violation_mgr.save_violation(
                            track_id, vehicle, frame,
                            plate_text, plate_img, self.fps
                        )
                        vehicle.violation_captured = True

                # Info overlay
                info_y = 30
                info_lines = [
                    f"Frame: {self.frame_num}/{total_frames} ({100*self.frame_num/total_frames:.1f}%)",
                    f"Vehicles: {len(tracks)}",
                    f"Violations: {len(self.violation_mgr.violations)}",
                    f"Yellow Box: {'DETECTED' if self.yellow_box_detector.current_mask is not None else 'NOT FOUND'}"
                ]

                for line in info_lines:
                    cv2.putText(
                        frame, line, (10, info_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2
                    )
                    cv2.putText(
                        frame, line, (10, info_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1
                    )
                    info_y += 30

                # Write frame
                out.write(frame)

                # Progress report mỗi 100 frames
                if self.frame_num % 100 == 0:
                    progress = 100 * self.frame_num / total_frames
                    print(f" Progress: {self.frame_num}/{total_frames} ({progress:.1f}%) | "
                          f"Vehicles: {len(tracks)} | Violations: {len(self.violation_mgr.violations)}")

        except KeyboardInterrupt:
            print("\n Processing interrupted by user")

        except Exception as e:
            print(f"\nError during processing: {str(e)}")
            import traceback
            traceback.print_exc()

        finally:
            # Cleanup
            self.cap.release()
            out.release()

            print("\n" + "="*70)
            print("PROCESSING COMPLETE!")
            print("="*70)
            print(f"Output video: {output_path}")
            print(f"Violations folder: {self.violation_mgr.dir}")
            print(f"Log file: {self.violation_mgr.log_path}")
            print(f"\n STATISTICS:")
            print(f"   Total frames processed: {self.frame_num}")
            print(f"   Total violations detected: {len(self.violation_mgr.violations)}")
            print(f"   Processing time: {self.frame_num/self.fps:.1f} seconds of video")

            if len(self.violation_mgr.violations) > 0:
                print(f"\n VIOLATIONS SUMMARY:")
                for i, v in enumerate(self.violation_mgr.violations, 1):
                    print(f"   {i}. [{v['id']}]")
                    print(f"      Plate: {v['plate_number']}")
                    print(f"      Type: {v['vehicle_type']}")
                    print(f"      Duration: {v['stopped_duration']}")
                    print(f"      Direction: {v['direction']}")
                    print(f"      Time: {v['timestamp']}")
            else:
                print(f"\n No violations detected!")

            print("="*70 + "\n")

        return self.violation_mgr.violations


In [ ]:

# COMPLETE WORKFLOW - Yellow Box Violation Detection + Email
import requests
from datetime import datetime
import time

API_URL = "https://traffic-violation-system-ei12.onrender.com/api"
VIDEO_PATH = "/content/a.mp4"
OUTPUT_PATH = "/content/output_violation_final.mp4"

# STEP 1: Wake up API & Verify Setup
print("\n1. Waking up API...")
try:
    response = requests.get(f"{API_URL}/health", timeout=60)
    if response.status_code == 200:
        print(" API is online!")
    else:
        print(" API returned unexpected status, retrying...")
        time.sleep(30)
        response = requests.get(f"{API_URL}/health", timeout=60)
except Exception as e:
    print(f" Cannot connect to API: {e}")
    print("  Please check Render service status")
    exit()

print("\n2. Checking database...")
response = requests.get(f"{API_URL}/vehicles", timeout=10)
if response.status_code == 200:
    vehicles = response.json()
    print(f" Vehicles in database: {len(vehicles)}")

    if len(vehicles) > 0:
        print(" Registered vehicles:")
        for v in vehicles[:5]:
            print(f" {v['plate_number']}: {v['owner_name']} ({v['email']})")
    else:
        print(" No vehicles in database!")
        print(" Add vehicles before processing video")
else:
    print(f" Cannot fetch vehicles: {response.status_code}")

print("\n3. Checking statistics...")
response = requests.get(f"{API_URL}/statistics", timeout=10)
if response.status_code == 200:
    stats = response.json()
    print(f" Current statistics:")
    print(f" Total vehicles: {stats['total_vehicles']}")
    print(f" Total violations: {stats['total_violations']}")
    print(f" Pending violations: {stats['pending_violations']}")
    print(f" Total fines: {stats['total_fines']:,.0f} VNĐ")

print("\n" + "=" * 70)
input("\n Setup complete! Press ENTER to start video processing...")

# STEP 2: Process Video with AI Detection
print("\n" + "=" * 70)
print("STEP 2: VIDEO PROCESSING")
print("=" * 70)

print(f"\n Input video: {VIDEO_PATH}")
print(f" Output video: {OUTPUT_PATH}")
print(f" API endpoint: {API_URL}")
print(f" Email notifications: ENABLED")
print()

try:
    # Initialize processor with API integration
    processor = MainProcessor(
        VIDEO_PATH,
        api_url=API_URL
    )

    # Process video
    print(" Starting video processing...\n")
    violations = processor.process(OUTPUT_PATH)


    print(f"\nPROCESSING SUMMARY:")
    print(f" Total violations detected: {len(violations)}")

    if len(violations) > 0:
        print(f"\n VIOLATION DETAILS:")

        for i, v in enumerate(violations, 1):
            print(f"\n   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
            print(f"   Violation #{i}")
            print(f"   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
            print(f"      ID: {v['id']}")
            print(f"      Plate: {v['plate_number']}")
            print(f"      Vehicle: {v['vehicle_type']}")
            print(f"      Location: {v['location']}")
            print(f"      Time: {v['timestamp']}")
            print(f"      Duration: {v['stopped_duration']}")
            print(f"      Direction: {v['direction']}")
            print(f"      Images:")
            print(f"         - Vehicle: {v['vehicle_image']}")
            print(f"         - Plate: {v['plate_image']}")

            # Email status
            if v.get('api_sent'):
                print(f" Sent to API: YES")
                print(f" Email notification: QUEUED")
                print(f" Check Gmail in 1-2 minutes")
                print(f" Look in Spam folder if not in Inbox")
            else:
                print(f" Sent to API: NO")
                print(f" Email notification: NOT SENT")
    else:
        print(f"\n No violations detected in this video!")

    # Email Verification
    if len(violations) > 0 and any(v.get('api_sent') for v in violations):
        print("\n EMAIL NOTIFICATION STATUS:")
        print(" Emails are being sent in background...")
        print(" This may take 1-3 minutes")
    else:
        print("\n No emails sent (no violations or vehicles not in database)")

    print("\n Checking violations in database...")
    time.sleep(2)

    response = requests.get(f"{API_URL}/violations", timeout=10)
    if response.status_code == 200:
        db_violations = response.json()
        recent = db_violations[-len(violations):] if len(violations) > 0 else []

        print(f" Total violations in DB: {len(db_violations)}")

        if recent:
            print(f"\n Latest violations:")
            for v in recent:
                email_status = " SENT" if v.get('email_sent') else " NOT SENT"
                vehicle_status = " FOUND" if v.get('vehicle_id') else " NOT FOUND"
                print(f"  ID {v['id']}: {v['plate_number']}")
                print(f"  Email: {email_status} | Vehicle: {vehicle_status}")

    # Final summary
    print("\n" + "=" * 70)
    print(" ALL DONE!")
    print("=" * 70)
    print(f"\n Output video: {OUTPUT_PATH}")
    print(f" Violations folder: /content/violations_output")
    print(f" Email notifications: {'SENT' if any(v.get('api_sent') for v in violations) else 'NONE'}")
    print(f"\n Thank you for using Yellow Box Violation Detection System!")
    print("=" * 70 + "\n")

except FileNotFoundError:
    print(f"\n ERROR: Video file not found")
    print(f"   Path: {VIDEO_PATH}")
    print("   Please check the file path and try again.")

except Exception as e:
    print(f"\n UNEXPECTED ERROR:")
    print(f"   {str(e)}")
    import traceback
    traceback.print_exc()